In [1]:
import os
import cv2
import numpy as np
import shutil
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, confusion_matrix

# Define your hyperparameters
frames_folder = r'sugi\Frame sugi'  # Replace with the actual path to your frames
model_filename = r'Model_TM_withoutbanket\model_DenseNet201_Thermal_Lightoff.h5'  # Replace with your model's filename
target_labels = ['Left Log', 'Prone Left', 'Prone Right', 'Right Log', 'Supine']

# Load the trained model
model = load_model(model_filename)

# Define any preprocessing transformations here (resize, normalization, etc.)
def preprocess_frame(frame):
    frame = cv2.resize(frame, (224, 224))  # Replace with the input size of your model
    frame = frame.astype(np.float32) / 255.0
    frame = np.expand_dims(frame, axis=0)
    return frame

# Initialize predictions and ground truth lists
predictions = []
ground_truth = []

# Create folders for each class label if they don't exist
for label in target_labels:
    label_folder = os.path.join(frames_folder, label)
    if not os.path.exists(label_folder):
        os.makedirs(label_folder)

# Loop through each frame in the folder, make predictions, and organize into class folders
for frame_name in sorted(os.listdir(frames_folder)):
    frame_path = os.path.join(frames_folder, frame_name)
    
    # Check if the current item is a file and is an image
    if os.path.isfile(frame_path) and frame_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Read the frame
        frame = cv2.imread(frame_path)

        # Preprocess the frame
        frame = preprocess_frame(frame)

        # Make a prediction on the frame
        output = model.predict(frame)

        # Get the predicted label
        predicted_label = target_labels[np.argmax(output)]

        # Record the predicted label
        predictions.append(predicted_label)

        # TODO: You need to assign the correct ground truth label for each frame
        true_label = 'Your Ground Truth Here'  # Replace with actual ground truth for the frame
        ground_truth.append(true_label)

        # Define destination path
        dest_path = os.path.join(frames_folder, predicted_label, frame_name)

        # Move the frame to the corresponding label folder
        shutil.move(frame_path, dest_path)

# Find unique labels present in ground_truth
unique_labels = list(set(ground_truth))

# Compute the confusion matrix only for labels present in ground_truth
conf_matrix = confusion_matrix(ground_truth, predictions, labels=unique_labels)
accuracy = accuracy_score(ground_truth, predictions)

print("Frames have been organized into class folders.")
print("Accuracy:", accuracy)
print("Confusion Matrix:", conf_matrix)


1/1 [==============================] - 0s 23ms/step
Frames have been organized into class folders.
Accuracy: 0.0
Confusion Matrix: [[0]]
